In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from full_utils import *
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from photutils.detection import find_peaks
from scipy.optimize import curve_fit
import glob
import pylab as pl

plt.rc('font', family='serif')
plt.rcParams["mathtext.fontset"] = 'cm'

In [ ]:
d = 16.8

xpetal = np.array([]); ypetal = np.array([]);

for i in range(7):
    xpetal = np.concatenate( (xpetal, d + i*d/2. + np.arange(7-i)*d) )
    ypetal = np.concatenate( (ypetal, np.full(7-i, i*d*np.sqrt(3.)/2.)) )

xhole = np.array([])
yhole = np.array([])

for i in range(6):
    theta = i*np.pi/3.
    
    xhole = np.concatenate((xhole, np.cos(theta)*xpetal-np.sin(theta)*ypetal))
    yhole = np.concatenate((yhole, np.sin(theta)*xpetal+np.cos(theta)*ypetal))

fid_flag = np.zeros(xhole.size, dtype=bool)
for i in range(xhole.size):
    if i%28==6 or i%28==9 or i%28==23:
        fid_flag[i] = True

fid_flag[34] = False
fid_flag[40] = True

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!! Modify Fiber Location!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  
fid_flag[9] = False
fid_flag[28*3+9] = False

fid_flag[8] = True
fid_flag[28*3+8] = True
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


xouter_fid = np.array([100.083000, 125.083000, -125.083000, -100.083000, -100.083000, -125.083000, 125.083000, 100.083000])
youter_fid = np.array([86.651000, 43.349000, 43.349000, 86.651000, -86.651000, -43.349000, -43.349000, -86.651000])

xouter_fid = np.array([125.083000, 100.083000, 25.000, -25.000,  -100.083000, -125.083000, -125.083000, -100.083000, -25.000, 25.000, 100.083000 , 125.083000])
youter_fid = np.array([43.349000, 86.651000, 130.000, 130.000, 86.651000, 43.349000,-43.349000, -86.651000, -130.000, -130.000, -86.651000 , -43.349000])

xhole = np.concatenate( (xhole, xouter_fid) )
yhole = np.concatenate( (yhole, youter_fid) )
fid_flag = np.concatenate( (fid_flag, np.full(xouter_fid.size, True)) )

In [ ]:
x, y = xhole[fid_flag], yhole[fid_flag]

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

ax.scatter(xhole, yhole, marker='+', c='r', s=8)

for i in range(xhole.size):
    fc = 'none'
    if fid_flag[i]: fc = 'tab:green'
    if i > 167: fc = 'tab:orange'
    if i==40: fc ='tab:blue'
    c1 = plt.Circle((xhole[i], yhole[i]), 7, ec='k', fc='none', lw=0.5)
    ax.add_artist(c1)
    c1 = plt.Circle((xhole[i], yhole[i]), 7, ec='none', fc=fc, alpha=0.4)
    ax.add_artist(c1)

c1 = plt.Circle((0,0), 7, ec='k', fc='gray', lw=0.5, alpha=0.2)
ax.add_artist(c1)

ax.set_xlim(-150., 150.); ax.set_ylim(-150., 150.)
ax.set_aspect('equal')

ax.set_xlabel('X [mm]', fontsize=14, labelpad=10)
ax.set_ylabel('Y [mm]', fontsize=14)

plt.tight_layout()

In [ ]:
im = fits.getdata('./FocusAdjust_Dark_108890_251224_0.fits')[::-1,:]

print(im.shape)

In [ ]:
npeaks = x.size
threshold = 1e3
box_size = 40

peak_table_raw = find_peaks(im, threshold=threshold, box_size=box_size)#, npeaks=npeaks)
peak_table = dedupe_peaks_kdtree(peak_table_raw, min_dist=40)
xf, yf = peak_table['x_peak'].data, peak_table['y_peak'].data

print(npeaks, xf.size)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(xf, yf, c='k', marker='.', s=5)

ax.set_aspect('equal')

In [ ]:
nwindow = 40
xobs = np.zeros(xf.size); yobs = np.copy(xobs)

im_obs = np.zeros((xf.size, 2*nwindow, 2*nwindow))
im_peak = np.zeros(xf.size)

for ifiber in range(xf.size):
    ipredict = np.argmin( np.abs(xccd_global[xf[ifiber]]-xccd_global) )
    jpredict = np.argmin( np.abs(yccd_global[yf[ifiber]]-yccd_global) )

    im_crop = im[jpredict-nwindow:jpredict+nwindow, ipredict-nwindow:ipredict+nwindow]

    im_obs[ifiber] = im_crop

    im_peak[ifiber] = im_crop.max()

    x_crop = xccd_global[ipredict-nwindow:ipredict+nwindow]
    y_crop = yccd_global[jpredict-nwindow:jpredict+nwindow]

    xobs[ifiber], yobs[ifiber] = com(im_crop, x_crop, y_crop)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,8))

xobs_temp, yobs_temp = transform(xobs, yobs, camera2focal_coeff)

for i in range(xobs.size):
    ax[0].scatter(xobs_temp[i], yobs_temp[i], c='k', marker='.')
    ax[0].text(xobs_temp[i]+3, yobs_temp[i]+3, str(i))

#ax[0].set_xlim(xccd_global.min(), xccd_global.max())
#ax[0].set_ylim(yccd_global.min(), yccd_global.max())
#ax[0].set_xlim(-12., 12.)
#ax[0].set_ylim(-10., 10.)


for i in range(x.size):
    c1 = plt.Circle((x[i], y[i]), 7, fc='none', ec='k')
    ax[1].add_artist(c1)
    #ax[1].text(x[i], y[i], str(i), ha='center', va='center')


for i in range(2):
    ax[i].set_xlim(-150, 150)
    ax[i].set_ylim(-150, 150)
    ax[i].set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(3, 6, figsize=(18, 9))

nax = ax.flatten()

for i in range(18):
    nax[i].imshow(np.log10(im_obs[i]), origin='lower', cmap='gray', vmin=2.5, vmax=4.5)
    print(im_obs[i].max())

In [ ]:
imatch, theta_guess, (xoff_guess, yoff_guess) = Match_Fiber(x, y, xobs, yobs, nbuffer=20)
xobs_match = xobs[imatch]; yobs_match = yobs[imatch]

print(theta_guess)
print(xoff_guess, yoff_guess)

In [ ]:
xobs_temp, yobs_temp = transform(xobs, yobs, camera2focal_coeff)

In [ ]:
nhunt = 720
theta_grid = np.linspace(0., 2.*np.pi, nhunt)
dd_sum = np.zeros(nhunt)
for ihunt, theta_temp in enumerate(theta_grid):
    xobs_rot = np.cos(theta_temp)*xobs_temp - np.sin(theta_temp)*yobs_temp
    yobs_rot = np.sin(theta_temp)*xobs_temp + np.cos(theta_temp)*yobs_temp
    obs_flag = np.concatenate( (np.full(xobs_temp.size, 0), np.full(xobs_temp.size, 1)) )
    pos_tot = np.concatenate( (np.vstack((x, y)).T
                                , np.vstack((xobs_rot, yobs_rot)).T) )

    tree = cKDTree(pos_tot)
    dd, ii = tree.query(pos_tot, k=8)

    for ipeak in range(xobs.size):
        dd_sum[ihunt] += dd[ipeak][obs_flag[ii[ipeak]] == 1].min()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.plot(theta_grid, dd_sum)

In [ ]:
add = 0.

xobs_rot = np.cos(theta_guess+add)*xobs_temp - np.sin(theta_guess+add)*yobs_temp
yobs_rot = np.sin(theta_guess+add)*xobs_temp + np.cos(theta_guess+add)*yobs_temp

In [ ]:
ngrid = 80
offset_grid = np.linspace(-10., 10., ngrid)
dsum_temp = np.zeros((ngrid, ngrid))
for i in range(ngrid):
    for j in range(ngrid):
        tree = cKDTree(np.c_[xobs_rot+offset_grid[i], yobs_rot+offset_grid[j]])
        d, _ = tree.query(np.c_[x, y], k=1)

        dsum_temp[i,j] = d.sum()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

im = ax.imshow(dsum_temp, origin='lower', cmap='jet', extent = [-10.-0.5, 10.+0.5, -10.-0.5, 10.+0.5])
fig.colorbar(im)

In [ ]:
imin, jmin = np.unravel_index(dsum_temp.argmin(), dsum_temp.shape)

xobs_rot += offset_grid[imin]
yobs_rot += offset_grid[jmin]

print(offset_grid[imin], offset_grid[jmin])

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(x, y, c='k', marker='.')

for i in range(xobs_rot.size):
    #c1 = plt.Circle((xobs_temp[i], yobs_temp[i]), 3, fc='none', ec='r', ls='--')
    #ax.add_artist(c1)
    
    c1 = plt.Circle((xobs_rot[i], yobs_rot[i]), 3, fc='none', ec='b', ls='--')
    ax.add_artist(c1)

ax.set_xlim(-150., 150.)
ax.set_ylim(-150., 150.)
ax.set_aspect('equal')

In [ ]:
xfit = np.concatenate((xobs_match, xobs_match))
yfit = np.concatenate((yobs_match, yobs_match))
ccd_fit = np.concatenate((x, y))


inv_popt_obs, _ = curve_fit(transform_polynomial, (xfit, yfit), ccd_fit
                       , maxfev=20000
                       , p0=[
                           camera2focal_coeff[0],
                           theta_guess,
                           -xoff_guess, -yoff_guess,
                           *camera2focal_coeff[4:]
                            ]
                       )
"""
inv_popt_obs, _ = curve_fit(distrad_SingleTelescope, (xfit, yfit), ccd_fit
                       , maxfev=20000
                       , p0=[
                           camera2focal_coeff[0],
                           theta_guess,
                           -xoff_guess, -yoff_guess,
                           *camera2focal_coeff[4:7]
                            ]
                       )
"""
print(inv_popt_obs)

In [ ]:
xfocal_obs, yfocal_obs = transform(xobs_match, yobs_match, inv_popt_obs)

dx = xfocal_obs-x
dy = yfocal_obs-y

dr = np.sqrt(dx**2 + dy**2)

for i in range(xfocal_obs.size):
    
    print(i, xfocal_obs[i], yfocal_obs[i], x[i], y[i], dx[i]*1e3, dy[i]*1e3, dr[i]*1e3, im_peak[imatch][i])

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(im_peak[imatch], dr*1e3, c='k')

ax.set_xlim(5e2, 8e4)
ax.set_ylim(0., 3e2)
ax.set_xscale('log')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))

ax[0].scatter(xobs_match, yobs_match, c=dx, cmap='jet', marker='s', s=50)
ax[1].scatter(xobs_match, yobs_match, c=dy, cmap='jet', marker='s', s=50)
ax[2].scatter(xobs_match, yobs_match, c=np.sqrt(dx**2 + dy**2), cmap='jet', marker='s', s=50)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(6,6))

sc = ax.scatter(xobs_match, yobs_match,
                c=np.sqrt(dx**2 + dy**2)*1e3,
                cmap='jet', marker='s', s=100)

ax.set_xlim(-20., 20.)
ax.set_ylim(-20., 20.)
ax.set_aspect('equal')

for i in range(3):
    c1 = plt.Circle((0,0), 5*(i+1), fc='none', ec='gray', lw=1, ls=':')
    ax.add_artist(c1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="4%", pad=0.05)  # size/pad는 취향대로
cbar = fig.colorbar(sc, cax=cax)

cbar.ax.tick_params(length=6, width=1, labelsize=14)
cbar.set_label(r'Accuracy [$\mu\mathrm{m}$]', fontsize=18, labelpad=10)


ax.axhline(0., 0., 1., c='gray', ls='--')
ax.axvline(0., 0., 1., c='gray', ls='--')

ax.tick_params(direction='in', labelsize=15, length=8, width=1)

ax.set_xlabel(r'$X_{\mathrm{Camera}}$ [mm]', fontsize=18, labelpad=10)
ax.set_ylabel(r'$Y_{\mathrm{Camera}}$ [mm]', fontsize=18, labelpad=10)

for i in range(3):
    c1 = plt.Circle((0,0), 5*(i+1), fc='none', ec='gray', lw=1, ls=':')
    ax.add_artist(c1)

plt.tight_layout()

#plt.savefig('../figures/251219/accuracy.png', dpi=200)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

for i in range(xobs_match.size):
    c1 = plt.Circle((xfocal_obs[i], yfocal_obs[i]), 4., fc='none', ec='b', ls='--')
    ax.add_artist(c1)

    #ax.text(xfocal_obs[i]+2, yfocal_obs[i]+2, str(i+1), color='b', fontsize=15)

ax.scatter(x, y, c='k', marker='.')


from matplotlib.lines import Line2D

custom_lines = [Line2D([0], [0], color='k', ls='none', marker='o', lw=3),
                Line2D([0], [0], color='b', ls='--', lw=3)]

ax.legend(custom_lines, ['Expected', 'Observed'], frameon=False, fontsize=18, loc='upper right')

ax.set_xlim(-170, 170)
ax.set_ylim(-170, 170)
ax.set_aspect('equal')

ax.tick_params(axis='x', direction='in', labelsize=15, length=8, width=1, pad=8)
ax.tick_params(axis='y', direction='in', labelsize=15, length=8, width=1)

ax.set_xlabel(r'$X_{\mathrm{focal}}$ [mm]', fontsize=20, labelpad=10)
ax.set_ylabel(r'$Y_{\mathrm{focal}}$ [mm]', fontsize=20, labelpad=10)

plt.tight_layout()
#plt.savefig('../figures/251219/loc.png', dpi=200)

In [ ]:
inv_popt_obs

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))

width = 2.5

for i in range(18):

    extent = [xobs[imatch][i]-width, xobs[imatch][i]+width, yobs[imatch][i]-width, yobs[imatch][i]+width]

    ax.imshow(np.log10(im_obs[imatch][i]), origin='lower', cmap='gray', vmin=2.5, vmax=4.5, extent=extent)

    ax.text(xobs[imatch][i]-width+0.2, yobs[imatch][i]+width-1, f'{(dr[i]*1e3):.2f}', c='cyan', fontsize=15)

ax.axhline(0., 0., 1., c='gray', ls='--')
ax.axvline(0., 0., 1., c='gray', ls='--')

ax.set_xlim(-20., 20.)
ax.set_ylim(-20., 20.)

ax.tick_params(direction='in', labelsize=18, length=8, width=1)

ax.set_xlabel(r'$X_{\mathrm{Camera}}$ [mm]', fontsize=20, labelpad=10)
ax.set_ylabel(r'$Y_{\mathrm{Camera}}$ [mm]', fontsize=20, labelpad=10)

ax.set_aspect('equal')

for i in range(3):
    c1 = plt.Circle((0,0), 5*(i+1), fc='none', ec='r', lw=1, ls=':')
    #ax.add_artist(c1)


plt.tight_layout()
#plt.savefig('../figures/251219/fid_image.png', dpi=200)

In [ ]:
xiter, yiter = np.copy(xfocal_obs), np.copy(yfocal_obs)
print(xiter, yiter)
for ii in range(1):
    ccd_fit = np.concatenate((xiter, yiter))

    inv_popt_obs, _ = curve_fit(transform_polynomial, (xfit, yfit), ccd_fit
                        , maxfev=20000
                        , p0=[
                            camera2focal_coeff[0],
                            theta_guess,
                            -xoff_guess, -yoff_guess,
                            *camera2focal_coeff[4:]
                                ]
                        )
    
    xiter_new, yiter_new = transform(xobs_match, yobs_match, inv_popt_obs)

    dx = xiter_new - xiter
    dy = yiter_new - yiter

    print(dx, dy)

    xiter = xiter_new
    yiter = yiter_new

In [ ]:
print(xiter_new, yiter_new)

In [ ]:
npeaks = 18
threshold = 5e2
box_size = 40
nwindow = 40

x, y = -xhole[fid_flag], yhole[fid_flag]

nframe = 100
nfiber = 18

xobs = np.zeros((nframe, nfiber)); yobs = np.copy(xobs)

im_obs = np.zeros((nframe, nfiber, 2*nwindow, 2*nwindow))
im_peak = np.zeros((nframe, nfiber))

xfocal_obs = np.zeros_like(xobs); yfocal_obs = np.zeros_like(yobs)

sat_flag = np.zeros(nframe, dtype=bool)

for iframe in range(nframe):
    im = fits.getdata(f'/md/cruithne/MOS/SingleTelescope/data/251219/test{iframe}.fits')

    if im.max() >= 65534:
        sat_flag[iframe] = True

    peak_table_raw = find_peaks(im, threshold=threshold, box_size=box_size, npeaks=npeaks)
    peak_table = dedupe_peaks_kdtree(peak_table_raw, min_dist=40)
    xf, yf = peak_table['x_peak'].data, peak_table['y_peak'].data
    print(iframe, npeaks, xf.size)


    for ifiber in range(nfiber):
        ipredict = np.argmin( np.abs(xccd_global[xf[ifiber]]-xccd_global) )
        jpredict = np.argmin( np.abs(yccd_global[yf[ifiber]]-yccd_global) )

        im_crop = im[jpredict-nwindow:jpredict+nwindow, ipredict-nwindow:ipredict+nwindow]

        im_obs[iframe, ifiber] = im_crop

        im_peak[iframe, ifiber] = im_crop.max()

        x_crop = xccd_global[ipredict-nwindow:ipredict+nwindow]
        y_crop = yccd_global[jpredict-nwindow:jpredict+nwindow]

        xobs[iframe, ifiber], yobs[iframe, ifiber] = com(im_crop, x_crop, y_crop)

    imatch, theta_guess, (xoff_guess, yoff_guess) = Match_Fiber(x, y, xobs[iframe], yobs[iframe], nbuffer=20)
    xobs_match = xobs[iframe][imatch]; yobs_match = yobs[iframe][imatch]

    xobs_rot = np.cos(theta_guess+add)*xobs_temp - np.sin(theta_guess+add)*yobs_temp
    yobs_rot = np.sin(theta_guess+add)*xobs_temp + np.cos(theta_guess+add)*yobs_temp

    ngrid = 80
    offset_grid = np.linspace(-10., 10., ngrid)
    dsum_temp = np.zeros((ngrid, ngrid))
    for i in range(ngrid):
        for j in range(ngrid):
            tree = cKDTree(np.c_[xobs_rot+offset_grid[i], yobs_rot+offset_grid[j]])
            d, _ = tree.query(np.c_[x, y], k=1)

            dsum_temp[i,j] = d.sum()

    xobs_rot += offset_grid[imin]
    yobs_rot += offset_grid[jmin]

    xfit = np.concatenate((xobs_match, xobs_match))
    yfit = np.concatenate((yobs_match, yobs_match))
    ccd_fit = np.concatenate((x, y))

    
    inv_popt_obs, _ = curve_fit(transform_polynomial, (xfit, yfit), ccd_fit
                        , maxfev=20000
                        , p0=[
                            camera2focal_coeff[0],
                            theta_guess,
                            -xoff_guess, -yoff_guess,
                            *camera2focal_coeff[4:]
                                ]
                        )
    xfocal_obs[iframe], yfocal_obs[iframe] = transform(xobs_match, yobs_match, inv_popt_obs)
    """
    inv_popt_obs, _ = curve_fit(distrad_SingleTelescope, (xfit, yfit), ccd_fit
                        , maxfev=20000
                        , p0=[
                            camera2focal_coeff[0],
                            theta_guess,
                            -xoff_guess, -yoff_guess,
                            *camera2focal_coeff[4:7]
                                ]
                        )
    xfocal_obs[iframe], yfocal_obs[iframe] = transform_single(xobs_match, yobs_match, inv_popt_obs)
    """

In [ ]:
px, py = (xfocal_obs - np.median(xfocal_obs, axis=0)), (yfocal_obs - np.median(yfocal_obs, axis=0))
pr = np.sqrt(px**2 + py**2)

In [ ]:
sx = np.std(xfocal_obs, axis=0)
sy = np.std(yfocal_obs, axis=0)

sr = np.sqrt(sx**2 + sy**2)

print(sx*1e3)
print(sy*1e3)
print(sr*1e3)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

points, cs = scatter_contour(
    px.flatten()*1e3, py.flatten()*1e3, ax=ax,
    normalize='max',
    threshold=0.05,                 # 1% 이상부터 시작
    levels=40,
    histogram2d_args={'bins': 30},
    contour_args={'cmap' : 'jet', 'vmin' : 0., 'vmax' : 1.},
    plot_args={'c' : 'b', 'marker' : '.', 'ms' : 2}
)


from mpl_toolkits.axes_grid1 import make_axes_locatable

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3.5%", pad=0.05)
cb = fig.colorbar(cs, cax=cax)
cb.set_label('Normalized density', labelpad=15, fontsize=17)
cb.set_ticks(np.arange(1,11)*0.1)
cb.ax.tick_params(length=6, width=1, labelsize=13)

ax.set_xlim(-40., 40.)
ax.set_ylim(-40., 40.)

ax.axhline(0., 0., 1., c='gray', ls='--', lw=1)
ax.axvline(0., 0., 1., c='gray', ls='--', lw=1)

ax.set_aspect('equal')

for i in range(3):
    c1 = plt.Circle((0,0), 5*(i+1), fc='none', ec='k', lw=1, zorder=3, ls='--')
    ax.add_artist(c1)

ax.tick_params(direction='in', length=8, width=1, labelsize=15)
ax.set_xlabel(r'$X-\tilde{X}$ [$\mu\mathrm{m}$]', fontsize=20, labelpad=5)
ax.set_ylabel(r'$Y-\tilde{Y}$ [$\mu\mathrm{m}$]', fontsize=20, labelpad=5)

plt.tight_layout()

#plt.savefig('../figures/251219/precision.png', dpi=200)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

for i in range(nfiber):
    ax.plot(np.arange(100), im_peak[:,i]/im_peak[0,i]-1., c='k', lw=1)

    #print(im_peak[:,i])

ax.set_ylim(-0.25, 0.25)


ax.set_xlabel('Frame', fontsize=18)
ax.set_ylabel('Fractional change', fontsize=18)

ax.tick_params(direction='in', length=6, width=1, labelsize=16)

plt.tight_layout()
#plt.savefig('../figures/251219/peak_pixel.png', dpi=200)

In [ ]:
dx, dy = (xfocal_obs - x), (yfocal_obs - y)
dr = np.sqrt(dx**2 + dy**2)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

for i in range(nfiber):
    ax.plot(np.arange(100), pr[:,i]*1e3, c='k', lw=1)

ax.set_ylabel('Fractional change', fontsize=18)
ax.tick_params(direction='in', length=6, width=1, labelsize=16)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(np.mean(im_peak[:,imatch], axis=0), sx*1e3, c='r', marker='o', label=r'$x$')
ax.scatter(np.mean(im_peak[:,imatch], axis=0), sy*1e3, c='b', marker='o', label=r'$y$')

ax.set_xlim(5e2, 8e4)
ax.set_ylim(0., 20)
ax.set_xscale('log')

ax.tick_params(which='major', direction='in', length=8, width=1, labelsize=15)
ax.tick_params(which='minor', direction='in', length=4, width=1, labelsize=15)

ax.set_xlabel(r'Peak value', fontsize=18)
ax.set_ylabel(r'$\sigma$ [$\mu\mathrm{m}$]', fontsize=22, labelpad=10)

ax.legend(fontsize=18, frameon=True)

ax.set_box_aspect(1)

plt.tight_layout()

plt.savefig('../figures/251219/precision_peakval.png', dpi=200)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))

ax[0].scatter(xfocal_obs, yfocal_obs, c=dx, cmap='jet', marker='s', s=50)
ax[1].scatter(xfocal_obs, yfocal_obs, c=dy, cmap='jet', marker='s', s=50)
ax[2].scatter(xfocal_obs, yfocal_obs, c=dr, cmap='jet', marker='s', s=50)

for i in range(3):
    ax[i].tick_params(direction='in', length=8, width=1, labelsize=15)

    ax[i].set_xlim(-150., 150.)
    ax[i].set_ylim(-150., 150.)

    ax[i].set_aspect('equal')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))

ax[0].scatter(xfocal_obs, yfocal_obs, c=dx, cmap='jet', marker='s', s=50)
ax[1].scatter(xfocal_obs, yfocal_obs, c=dy, cmap='jet', marker='s', s=50)
ax[2].scatter(xfocal_obs, yfocal_obs, c=dr, cmap='jet', marker='s', s=50)

for i in range(3):
    ax[i].tick_params(direction='in', length=8, width=1, labelsize=15)

    ax[i].set_xlim(-150., 150.)
    ax[i].set_ylim(-150., 150.)

    ax[i].set_aspect('equal')